In [24]:
import pandas as pd
import random
import time
import pymongo
from pymongo import MongoClient
from parsel import Selector
from datetime import datetime
from time import sleep
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import itertools

# Connecting ot db

connection = MongoClient("mongodb://localhost:27017")
db = connection["IndeedJobPostDB"]

chrome_options = Options()

PROXY = "" # HOST:PORT

chrome_options = uc.ChromeOptions()
chrome_options.add_argument(f'--proxy-server={PROXY}')

driver = uc.Chrome(executable_path=r'chromedriver', options=chrome_options)

position = input('Enter your preferred skill: ').strip()
place = input('Enter the location: ').strip()
no_of_pages = int(input('Enter the number of pages to scrape: '))

Enter your preferred skill: Restaurant
Enter the location: New York, NY
Enter the number of pages to scrape: 62


In [30]:
job_data = []
for page in range(0, no_of_pages*10, 10):
    driver.get(f'https://www.indeed.com/jobs?q={position}&l={place}&sort=date&start={page}')
    time.sleep(random.uniform(8.5, 10.9))
    
    try:
        close = driver.find_element(By.XPATH, '//div[@class="icl-CloseButton icl-Modal-close"]')
        close.click()
    except:
        pass
    page_source = driver.page_source

    soup = BeautifulSoup(page_source, features="html.parser")
    divs = soup.find_all('div', 'slider_container css-77eoo7 eu4oa1w0')
    #jobs = driver.find_elements(By.CLASS_NAME, 'slider_container css-77eoo7 eu4oa1w0')
    #jobs = driver.find_elements(By.XPATH, "//div[contains(@class, 'slider_container css-77eoo7 eu4oa1w0')]")
    print(len(jobs))
    for item in divs:
        
        # Getting the Job Title
        title = item.find('a').text.strip()
        #print(title)
        type_of_job = ''
        salary = ''
        schedule = ''
        # Getting the job type
        try:
            all_metadata = item.find('div', 'heading6 tapItem-gutter metadataContainer noJEMChips salaryOnly')
            print(all_metadata.text)
            rows =[i.get_text(strip=True) for i  in all_metadata.find_all('div', 'metadata') if i.get_text(strip=True)!=""]
            print(rows)
            for element in rows:
                if '$' in element:
                    salary = element.strip()
                    rows.remove(element)
                    print(salary)
                else:
                    salary = ''
            for element in rows:
                if 'time' in element:
                    if '+1' in element:
                        type_of_job = 'Full-time, Part-time'
                    else:
                        type_of_job = element
                    rows.remove(element)
                    print(type_of_job)
                else:
                    type_of_job = ''
            if len(rows) > 0:
                schedule = rows[0]
                print(schedule)
            else:
                schedule = ''
        except:
            try:
                salary = item.find('div', 'attribute_snippet').text.strip()
                type_of_job = ''
            except:
                salary = ''

        try:
            company = item.find('span', 'companyName').text.strip()
        except:
            company = ''

        #Getting the Location
        try:
            location = item.find('div', 'companyLocation').text.strip()
        except:
            location = ''
 
        #Getting the link
        try:
            link = 'https://www.indeed.com/' + item.h2.a.get('href')
        except:
            link = ''
        
        # Date Posted
        try:
            date_posted = item.find('span', 'date').text.strip()
        except:
            date_posted = ''

        # Current Date
        try: 
            current_date = datetime.today().strftime('%y-%m-%d')
        except:
            current_date = ''
        
        try:
            description  = item.find('div', 'job-snippet').text.strip().replace('\n', ' ')
        except:
            description = ''

        data = {
            'Title' : title,
            'Job_Type' : type_of_job,
            'Company' : company,
            'Salary' : salary,
            'Schedule': schedule,
            'Location' : location,
            'Date_Posted' : date_posted.removeprefix('Posted'),
            'Current_Date' : current_date,
            'Summary' : description,
            'Apply_link' : link
        }
        job_data.append(data)
        print('[*] Saving')

df_NY = pd.DataFrame(job_data)

15
Estimated $22.9K - $29K a year
['Estimated $22.9K - $29K a year']
Estimated $22.9K - $29K a year
[*] Saving
Estimated $19.2K - $24.3K a year
['Estimated $19.2K - $24.3K a year']
Estimated $19.2K - $24.3K a year
[*] Saving
$15.00 - $20.25 an hour
['$15.00 - $20.25 an hour']
$15.00 - $20.25 an hour
[*] Saving
From $14.50 an hour
['From $14.50 an hour']
From $14.50 an hour
[*] Saving
$45,000 - $55,000 a year
['$45,000 - $55,000 a year']
$45,000 - $55,000 a year
[*] Saving
Estimated $57.9K - $73.3K a year
['Estimated $57.9K - $73.3K a year']
Estimated $57.9K - $73.3K a year
[*] Saving
$17 - $20 an hour
['$17 - $20 an hour']
$17 - $20 an hour
[*] Saving
$60,000 - $75,115 a year
['$60,000 - $75,115 a year']
$60,000 - $75,115 a year
[*] Saving
$17 - $20 an hour
['$17 - $20 an hour']
$17 - $20 an hour
[*] Saving
Estimated $30.4K - $38.5K a year
['Estimated $30.4K - $38.5K a year']
Estimated $30.4K - $38.5K a year
[*] Saving
Estimated $30.4K - $38.4K a year
['Estimated $30.4K - $38.4K a year

15
$18 - $25 an hour
['$18 - $25 an hour']
$18 - $25 an hour
[*] Saving
Estimated $26.2K - $33.2K a year
['Estimated $26.2K - $33.2K a year']
Estimated $26.2K - $33.2K a year
[*] Saving
$30 an hour
['$30 an hour']
$30 an hour
[*] Saving
$17.51 an hour
['$17.51 an hour']
$17.51 an hour
[*] Saving
$18 - $25 an hour
['$18 - $25 an hour']
$18 - $25 an hour
[*] Saving
$18 - $25 an hour
['$18 - $25 an hour']
$18 - $25 an hour
[*] Saving
Estimated $24.2K - $30.7K a year
['Estimated $24.2K - $30.7K a year']
Estimated $24.2K - $30.7K a year
[*] Saving
$15 - $20 an hour
['$15 - $20 an hour']
$15 - $20 an hour
[*] Saving
$16.50 an hour
['$16.50 an hour']
$16.50 an hour
[*] Saving
$16 - $20 an hour
['$16 - $20 an hour']
$16 - $20 an hour
[*] Saving
$15 an hour
['$15 an hour']
$15 an hour
[*] Saving
Estimated $25.4K - $32.2K a year
['Estimated $25.4K - $32.2K a year']
Estimated $25.4K - $32.2K a year
[*] Saving
$17 an hour
['$17 an hour']
$17 an hour
[*] Saving
Estimated $23K - $29.1K a year
['Esti

15
Estimated $25.3K - $32.1K a year
['Estimated $25.3K - $32.1K a year']
Estimated $25.3K - $32.1K a year
[*] Saving
$16 - $17 an hour
['$16 - $17 an hour']
$16 - $17 an hour
[*] Saving
Estimated $21.6K - $27.3K a year
['Estimated $21.6K - $27.3K a year']
Estimated $21.6K - $27.3K a year
[*] Saving
$20 - $28 an hour
['$20 - $28 an hour']
$20 - $28 an hour
[*] Saving
$18 an hour
['$18 an hour']
$18 an hour
[*] Saving
From $100,000 a year
['From $100,000 a year']
From $100,000 a year
[*] Saving
$20 an hour
['$20 an hour']
$20 an hour
[*] Saving
Estimated $51.9K - $65.7K a year
['Estimated $51.9K - $65.7K a year']
Estimated $51.9K - $65.7K a year
[*] Saving
Estimated $31.2K - $39.5K a year
['Estimated $31.2K - $39.5K a year']
Estimated $31.2K - $39.5K a year
[*] Saving
$15 - $16 an hour
['$15 - $16 an hour']
$15 - $16 an hour
[*] Saving
$16 - $18 an hour
['$16 - $18 an hour']
$16 - $18 an hour
[*] Saving
$23 - $33 an hour
['$23 - $33 an hour']
$23 - $33 an hour
[*] Saving
$15 - $25 an hou

15
Estimated $31.6K - $40K a year
['Estimated $31.6K - $40K a year']
Estimated $31.6K - $40K a year
[*] Saving

[]
[*] Saving
$15 - $30 an hour
['$15 - $30 an hour']
$15 - $30 an hour
[*] Saving
From $13 an hour
['From $13 an hour']
From $13 an hour
[*] Saving
Estimated $28.8K - $36.4K a year
['Estimated $28.8K - $36.4K a year']
Estimated $28.8K - $36.4K a year
[*] Saving
$50,000 - $55,000 a year
['$50,000 - $55,000 a year']
$50,000 - $55,000 a year
[*] Saving
$55,000 - $60,000 a year
['$55,000 - $60,000 a year']
$55,000 - $60,000 a year
[*] Saving

[]
[*] Saving
Estimated $28.5K - $36.1K a year
['Estimated $28.5K - $36.1K a year']
Estimated $28.5K - $36.1K a year
[*] Saving
Estimated $33.4K - $42.3K a year
['Estimated $33.4K - $42.3K a year']
Estimated $33.4K - $42.3K a year
[*] Saving
Estimated $28.5K - $36.1K a year
['Estimated $28.5K - $36.1K a year']
Estimated $28.5K - $36.1K a year
[*] Saving
Estimated $29.1K - $36.8K a year
['Estimated $29.1K - $36.8K a year']
Estimated $29.1K -

15
$18 - $20 an hour
['$18 - $20 an hour']
$18 - $20 an hour
[*] Saving
From $70,000 a year
['From $70,000 a year']
From $70,000 a year
[*] Saving
$17 - $22 an hour
['$17 - $22 an hour']
$17 - $22 an hour
[*] Saving
Estimated $37.2K - $47.1K a year
['Estimated $37.2K - $47.1K a year']
Estimated $37.2K - $47.1K a year
[*] Saving
$70,000 a year
['$70,000 a year']
$70,000 a year
[*] Saving
$17 - $20 an hour
['$17 - $20 an hour']
$17 - $20 an hour
[*] Saving
$16.50 an hour
['$16.50 an hour']
$16.50 an hour
[*] Saving
$76,000 - $112,000 a year
['$76,000 - $112,000 a year']
$76,000 - $112,000 a year
[*] Saving
$17 an hour
['$17 an hour']
$17 an hour
[*] Saving
$19 - $21 an hour
['$19 - $21 an hour']
$19 - $21 an hour
[*] Saving
Estimated $32K - $40.5K a year
['Estimated $32K - $40.5K a year']
Estimated $32K - $40.5K a year
[*] Saving
$19 - $21 an hour
['$19 - $21 an hour']
$19 - $21 an hour
[*] Saving

[]
[*] Saving
Estimated $27.3K - $34.6K a year
['Estimated $27.3K - $34.6K a year']
Estima

15
$27 an hour
['$27 an hour']
$27 an hour
[*] Saving
$17.50 an hour
['$17.50 an hour']
$17.50 an hour
[*] Saving
$15 an hour
['$15 an hour']
$15 an hour
[*] Saving
$17.25 an hour
['$17.25 an hour']
$17.25 an hour
[*] Saving
$19 - $23 an hour
['$19 - $23 an hour']
$19 - $23 an hour
[*] Saving
$15 an hour
['$15 an hour']
$15 an hour
[*] Saving
$110,000 - $130,000 a year
['$110,000 - $130,000 a year']
$110,000 - $130,000 a year
[*] Saving
$15 - $20 an hour
['$15 - $20 an hour']
$15 - $20 an hour
[*] Saving
$21 an hour
['$21 an hour']
$21 an hour
[*] Saving
$70,000 - $75,000 a year
['$70,000 - $75,000 a year']
$70,000 - $75,000 a year
[*] Saving
$20 - $22 an hour
['$20 - $22 an hour']
$20 - $22 an hour
[*] Saving
$19 - $22 an hour
['$19 - $22 an hour']
$19 - $22 an hour
[*] Saving
$22 - $26 an hour
['$22 - $26 an hour']
$22 - $26 an hour
[*] Saving
Estimated $32.6K - $41.2K a year
['Estimated $32.6K - $41.2K a year']
Estimated $32.6K - $41.2K a year
[*] Saving
$20 - $22 an hour
['$20 - $2

15
$75,000 - $80,000 a year
['$75,000 - $80,000 a year']
$75,000 - $80,000 a year
[*] Saving
$15 - $20 an hour
['$15 - $20 an hour']
$15 - $20 an hour
[*] Saving
$56,000 - $62,000 a year
['$56,000 - $62,000 a year']
$56,000 - $62,000 a year
[*] Saving
Estimated $61.4K - $77.8K a year
['Estimated $61.4K - $77.8K a year']
Estimated $61.4K - $77.8K a year
[*] Saving

[]
[*] Saving
$31,000 - $32,837 a year
['$31,000 - $32,837 a year']
$31,000 - $32,837 a year
[*] Saving
$70,000 - $78,000 a year
['$70,000 - $78,000 a year']
$70,000 - $78,000 a year
[*] Saving
$68,000 - $75,000 a year
['$68,000 - $75,000 a year']
$68,000 - $75,000 a year
[*] Saving
$75,000 - $80,000 a year
['$75,000 - $80,000 a year']
$75,000 - $80,000 a year
[*] Saving
Estimated $71.9K - $91.1K a year
['Estimated $71.9K - $91.1K a year']
Estimated $71.9K - $91.1K a year
[*] Saving
$60,000 a year
['$60,000 a year']
$60,000 a year
[*] Saving
$13 - $20 an hour
['$13 - $20 an hour']
$13 - $20 an hour
[*] Saving
Estimated $31.5K

15
$15 - $50 an hour
['$15 - $50 an hour']
$15 - $50 an hour
[*] Saving
Estimated $34.3K - $43.4K a year
['Estimated $34.3K - $43.4K a year']
Estimated $34.3K - $43.4K a year
[*] Saving
$15 - $17 an hour
['$15 - $17 an hour']
$15 - $17 an hour
[*] Saving
$30.03 - $35.52 an hour
['$30.03 - $35.52 an hour']
$30.03 - $35.52 an hour
[*] Saving
Estimated $28.7K - $36.3K a year
['Estimated $28.7K - $36.3K a year']
Estimated $28.7K - $36.3K a year
[*] Saving
$15 - $25 an hour
['$15 - $25 an hour']
$15 - $25 an hour
[*] Saving
$15 an hour
['$15 an hour']
$15 an hour
[*] Saving
$22.00 - $27.42 an hour
['$22.00 - $27.42 an hour']
$22.00 - $27.42 an hour
[*] Saving
$17.00 - $20.40 an hour
['$17.00 - $20.40 an hour']
$17.00 - $20.40 an hour
[*] Saving
$15 an hour
['$15 an hour']
$15 an hour
[*] Saving
$15 - $20 an hour
['$15 - $20 an hour']
$15 - $20 an hour
[*] Saving
$17 - $18 an hour
['$17 - $18 an hour']
$17 - $18 an hour
[*] Saving
From $62,000 a year
['From $62,000 a year']
From $62,000 a ye

15
Estimated $40.6K - $51.4K a year
['Estimated $40.6K - $51.4K a year']
Estimated $40.6K - $51.4K a year
[*] Saving
Estimated $29.8K - $37.7K a year
['Estimated $29.8K - $37.7K a year']
Estimated $29.8K - $37.7K a year
[*] Saving
$85,000 - $110,000 a year
['$85,000 - $110,000 a year']
$85,000 - $110,000 a year
[*] Saving
Estimated $47.1K - $59.7K a year
['Estimated $47.1K - $59.7K a year']
Estimated $47.1K - $59.7K a year
[*] Saving
Estimated $53.1K - $67.2K a year
['Estimated $53.1K - $67.2K a year']
Estimated $53.1K - $67.2K a year
[*] Saving
$25 an hour
['$25 an hour']
$25 an hour
[*] Saving
Estimated $28.8K - $36.4K a year
['Estimated $28.8K - $36.4K a year']
Estimated $28.8K - $36.4K a year
[*] Saving
$60,000 - $93,000 a year
['$60,000 - $93,000 a year']
$60,000 - $93,000 a year
[*] Saving
$20.50 an hour
['$20.50 an hour']
$20.50 an hour
[*] Saving
Estimated $29.8K - $37.7K a year
['Estimated $29.8K - $37.7K a year']
Estimated $29.8K - $37.7K a year
[*] Saving
$17 - $19 an hour
[

In [31]:
df_NY['Job_Type'].head(30)

0     
1     
2     
3     
4     
5     
6     
7     
8     
9     
10    
11    
12    
13    
14    
15    
16    
17    
18    
19    
20    
21    
22    
23    
24    
25    
26    
27    
28    
29    
Name: Job_Type, dtype: object

In [32]:
df_NY['Salary'].head(70)

0       Estimated $22.9K - $29K a year
1     Estimated $19.2K - $24.3K a year
2              $15.00 - $20.25 an hour
3                  From $14.50 an hour
4             $45,000 - $55,000 a year
                    ...               
65           $90,000 - $100,000 a year
66            $65,000 - $80,000 a year
67            $60,000 - $65,000 a year
68                         $15 an hour
69                   $21 - $24 an hour
Name: Salary, Length: 70, dtype: object

In [33]:
data = df_NY.to_dict(orient="records")
db.Job_Posts.insert_many(data)

In [14]:
df["Summary"][6]

"Beverage-wise, K'Far will offer a full coffee drink menu and cocktails with nods to Isreal.\nOpen schedule availability is preferred, and open minds are a must."

In [15]:
b

'Full-time'